In [139]:
import ctypes
import numpy as np
import math
import time

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

class DacQueue:
    '''
    A queue for patterns sent to the dac. 
    Performs smart stitching between patterns.
    '''
    def __init__(self):
        # Dac object for this queue
        self.dac = Dac()
        # The last position of the last pattern (x,y)
        self.last_pos = (0,0)
    
    def submit(self, pat_pos, pat_col, angular_density=20):
        '''
        Submits a new pattern to the dac, transitioning smoothly from the last one
        '''
        # Make the transition
        dist = np.sqrt(np.sum(np.power(self.last_pos-pat_pos[0,:], 2)))
        num_gap_points = int(dist*angular_density)
        gap_pos = np.linspace(self.last_pos, pat_pos[0,:], num_gap_points, dtype=np.float)
        gap_col = np.ones((num_gap_points, 3), dtype=np.float)/4
        print('gap_pos', gap_pos)
        
        # Prep the transition pattern
        gap_points, num_gap_points = self.prep_pattern(gap_pos, gap_col)
        
        # Prep the new pattern
        pat_points, num_pat_points = self.prep_pattern(pat_pos, pat_col)
        
        # Send the transition pattern to the dac
        self.write_frames(gap_points, num_gap_points)
        # Send the new pattern to the dac
        self.write_frames(pat_points, num_pat_points)
        # Set the last_position
        self.last_pos = pat_pos[-1, :]
        
    def prep_pattern(self, arr_pos, arr_col):
        '''
        1) Scales and color corrects 
        2) Converts from unit space to DAC coordinates
        3) Produces a DAC compatible frame and displays it
        '''
        # Force pass-by-value
        arr_pos = arr_pos.copy()
        arr_col = arr_col.copy()
        # Scale the pattern 
        arr_pos = position_corection(arr_pos)
        # Format the position array (-1.0-1.0) -> int(0-4095)
        arr_pos = (((arr_pos+1)/2.0)*self.dac.xy_max).astype(np.int32)

        # Perform color correction
        arr_col = color_correction(arr_col)
        # Format the color array (0-1.0) -> int(0-255)
        arr_col = (arr_col*255).astype(np.int32)
        # Fill a heliospoint arr with these values
        num_points = len(arr_pos)

        frameType = HeliosPoint * num_points
        points = frameType()
        # Fill the frame
        for idx in range(num_points):
            points[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                         int(arr_pos[idx, 1]), 
                                         int(arr_col[idx, 0]), 
                                         int(arr_col[idx, 1]),
                                         int(arr_col[idx, 2]),
                                         int(255))
        
        return points, num_points
            
    def write_frames(self, points, num_points):
        if(num_points < 1):
            return
        # Write the values out 
        status_attempts = 0
        max_attempts = np.inf
        # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
        while(status_attempts < max_attempts and self.dac.HeliosLib.GetStatus(0) != 1):
            status_attempts += 1
            print('SA: ', status_attempts)

        self.dac.HeliosLib.WriteFrame(0, 500, 0, ctypes.pointer(points), num_points)



In [140]:
def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    #arr_color = np.exp(np.log(2)*np.power(arr_color,1)) - 1
    # Scale to the lower cutoff (R, G, B)
    lower_cutoff = np.array([0.25, 0.09, 0.09])
    arr_color = arr_color*(1-lower_cutoff) + lower_cutoff
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
def make_circular(arr_pos, arr_color):
    return np.concatenate([arr_pos, arr_pos[::-1, :]]), np.concatenate([arr_color, arr_color[::-1, :]])


In [141]:
queue = DacQueue()

Found  1 Helios DACs


In [143]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

start = (0.1, 1.0, 0.1)
finish = (1.0, 0.1, 1.0)
smooth = np.linspace(start, finish, T//2)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

queue.submit(pos_arr, color_arr)

queue.submit(pos_arr[:, ::-1], color_arr)

queue.submit(pos_arr, color_arr)



gap_pos []
gap_pos [[ 1.00000000e+00 -2.44929360e-16]
 [ 9.62962963e-01  3.70370370e-02]
 [ 9.25925926e-01  7.40740741e-02]
 [ 8.88888889e-01  1.11111111e-01]
 [ 8.51851852e-01  1.48148148e-01]
 [ 8.14814815e-01  1.85185185e-01]
 [ 7.77777778e-01  2.22222222e-01]
 [ 7.40740741e-01  2.59259259e-01]
 [ 7.03703704e-01  2.96296296e-01]
 [ 6.66666667e-01  3.33333333e-01]
 [ 6.29629630e-01  3.70370370e-01]
 [ 5.92592593e-01  4.07407407e-01]
 [ 5.55555556e-01  4.44444444e-01]
 [ 5.18518519e-01  4.81481481e-01]
 [ 4.81481481e-01  5.18518519e-01]
 [ 4.44444444e-01  5.55555556e-01]
 [ 4.07407407e-01  5.92592593e-01]
 [ 3.70370370e-01  6.29629630e-01]
 [ 3.33333333e-01  6.66666667e-01]
 [ 2.96296296e-01  7.03703704e-01]
 [ 2.59259259e-01  7.40740741e-01]
 [ 2.22222222e-01  7.77777778e-01]
 [ 1.85185185e-01  8.14814815e-01]
 [ 1.48148148e-01  8.51851852e-01]
 [ 1.11111111e-01  8.88888889e-01]
 [ 7.40740741e-02  9.25925926e-01]
 [ 3.70370370e-02  9.62962963e-01]
 [ 0.00000000e+00  1.00000000e+00]]


SA:  901
SA:  902
SA:  903
SA:  904
SA:  905
SA:  906
SA:  907
SA:  908
SA:  909
SA:  910
SA:  911
SA:  912
SA:  913
SA:  914
SA:  915
SA:  916
SA:  917
SA:  918
SA:  919
SA:  920
SA:  921
SA:  922
SA:  923
SA:  924
SA:  925
SA:  926
SA:  927
SA:  928
SA:  929
SA:  930
SA:  931
SA:  932
SA:  933
SA:  934
SA:  935
SA:  936
SA:  937
SA:  938
SA:  939
SA:  940
SA:  941
SA:  942
SA:  943
SA:  944
SA:  945
SA:  946
SA:  947
SA:  948
SA:  949
SA:  950
SA:  951
SA:  952
SA:  953
SA:  954
SA:  955
SA:  956
SA:  957
SA:  958
SA:  959
SA:  960
SA:  961
SA:  962
SA:  963
SA:  964
SA:  965
SA:  966
SA:  967
SA:  968
SA:  969
SA:  970
SA:  971
SA:  972
SA:  973
SA:  974
SA:  975
SA:  976
SA:  977
SA:  978
SA:  979
SA:  980
SA:  981
SA:  982
SA:  983
SA:  984
SA:  985
SA:  1
SA:  2
SA:  3
SA:  4
SA:  5
SA:  6
SA:  7
SA:  8
SA:  9
SA:  10
SA:  11
SA:  12
SA:  13
SA:  14
SA:  15
SA:  16
SA:  17
SA:  18
SA:  19
SA:  20
SA:  21
SA:  22
SA:  23
SA:  24
SA:  25
SA:  26
SA:  27
SA:  28
SA:  29
SA:  30
SA: 

SA:  925
SA:  926
SA:  927
SA:  928
SA:  929
SA:  930
SA:  931
SA:  932
SA:  933
SA:  934
SA:  935
SA:  936
SA:  937
SA:  938
SA:  939
SA:  940
SA:  941
SA:  942
SA:  943
SA:  944
SA:  945
SA:  946
SA:  947
SA:  948
SA:  949
SA:  950
SA:  951
SA:  952
SA:  953
SA:  954
SA:  955
SA:  956
SA:  957
SA:  958
SA:  959
SA:  960
SA:  961
SA:  962
SA:  963
SA:  964
SA:  965
SA:  966
SA:  967
SA:  968
SA:  969
SA:  970
SA:  971
SA:  972
SA:  973
SA:  974
SA:  975
SA:  976
SA:  977
SA:  978
SA:  979
SA:  980
SA:  981
SA:  982
SA:  983
SA:  984
SA:  985
SA:  986
SA:  987
SA:  988
SA:  989
SA:  990
SA:  991
SA:  992
SA:  993
SA:  994
SA:  995
SA:  996
SA:  997
SA:  998
SA:  999
gap_pos [[-2.44929360e-16  1.00000000e+00]
 [ 3.70370370e-02  9.62962963e-01]
 [ 7.40740741e-02  9.25925926e-01]
 [ 1.11111111e-01  8.88888889e-01]
 [ 1.48148148e-01  8.51851852e-01]
 [ 1.85185185e-01  8.14814815e-01]
 [ 2.22222222e-01  7.77777778e-01]
 [ 2.59259259e-01  7.40740741e-01]
 [ 2.96296296e-01  7.03703704e-01]
 [

SA:  805
SA:  806
SA:  807
SA:  808
SA:  809
SA:  810
SA:  811
SA:  812
SA:  813
SA:  814
SA:  815
SA:  816
SA:  817
SA:  818
SA:  819
SA:  820
SA:  821
SA:  822
SA:  823
SA:  824
SA:  825
SA:  826
SA:  827
SA:  828
SA:  829
SA:  830
SA:  831
SA:  832
SA:  833
SA:  834
SA:  835
SA:  836
SA:  837
SA:  838
SA:  839
SA:  840
SA:  841
SA:  842
SA:  843
SA:  844
SA:  845
SA:  846
SA:  847
SA:  848
SA:  849
SA:  850
SA:  851
SA:  852
SA:  853
SA:  854
SA:  855
SA:  856
SA:  857
SA:  858
SA:  859
SA:  860
SA:  861
SA:  862
SA:  863
SA:  864
SA:  865
SA:  866
SA:  867
SA:  868
SA:  869
SA:  870
SA:  871
SA:  872
SA:  873
SA:  874
SA:  875
SA:  876
SA:  877
SA:  878
SA:  879
SA:  880
SA:  881
SA:  882
SA:  883
SA:  884
SA:  885
SA:  886
SA:  887
SA:  888
SA:  889
SA:  890
SA:  891
SA:  892
SA:  893
SA:  894
SA:  895
SA:  896
SA:  897
SA:  898
SA:  899
SA:  900
SA:  901
SA:  902
SA:  903
SA:  904
SA:  905
SA:  906
SA:  907
SA:  908
SA:  909
SA:  910
SA:  911
SA:  912
SA:  913
SA:  914
SA:  915
S

In [80]:
# Make a tri-colored circle
# Make a tricolor circle
T = 1000
pos_arr = np.zeros((T, 2))
col_arr = np.zeros((T, 3))

# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
pos_arr[:,0] = np.cos(theta)
pos_arr[:,1] = np.sin(theta)


# Tri color the circle
col_arr[theta < np.pi, 0] = 0
col_arr[theta < np.pi, 1] = 0
col_arr[theta < np.pi, 2] = 0.3
col_arr[theta < (2/3)*np.pi, 0] = 0
col_arr[theta < (2/3)*np.pi, 1] = 0.3
col_arr[theta < (2/3)*np.pi, 2] = 0
col_arr[theta < (1/3)*np.pi, 0] = 0.3
col_arr[theta < (1/3)*np.pi, 1] = 0
col_arr[theta < (1/3)*np.pi, 2] = 0

dacpos_batch(pos_arr, col_arr, dac)
print(pos_arr.dtype)

NameError: name 'dac' is not defined

In [81]:
def func(x):
    x = x + 1

x = 5
func(x)
print(x)

5


In [ ]:
scales = np.concatenate([np.linspace(1.0, 0.5, 20), np.linspace(0.5, 1, 20)])

import time
while(True):
    for scale in scales:
        new_pos_arr = scale*pos_arr
        dacpos_batch(new_pos_arr, col_arr, dac)
        #print(scale, new_pos_arr[0:4, 0])


In [ ]:
pos_list = [pos_arr, 0.5*pos_arr, 0.1*pos_arr]
col_list = [col_arr, col_arr, col_arr]
print('a: ', pos_list[0].shape, col_list[0].shape)
pos_list, col_list = stitch_patterns(pos_list, col_list)
print('b: ', pos_list[0].shape, col_list[0].shape)

while(True):
    for pos_arr, col_arr in zip(pos_list, col_list):
        print(pos_arr.shape, col_arr.shape)
        dacpos_batch(pos_arr, col_arr, dac)

In [ ]:
import cv2
def draw_bitmap(arr_pos, arr_color, dims=(400,400)):
    '''
    Draws a pattern as if we were drawing it on a wall
    '''
    # Copy the arrays so we don't change them 
    arr_pos = arr_pos.copy()
    arr_color = arr_color.copy()
    # Map the positions
    arr_pos[:,0] *= -1
    arr_pos = (((arr_pos+1)/2)*dims).astype(np.int)
    # Map the colors
    arr_color = arr_color[:,::-1]
    T = len(arr_pos)
    img = np.zeros(dims+(3,))
    # Draw points
    for t in range(1, T):
        cv2.line(img, arr_pos[t-1], arr_pos[t], color=arr_color[t-1], thickness=2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
    

In [ ]:
np.zeros((12, 3)).shape

In [ ]:
# Make three parallel lines
T_line = 100

arr_pos_list = []
arr_color_list = []
for c_idx, y in enumerate(range(-1, 2, 1)):
    print(c_idx, y)
    arr_pos_list.append(np.stack([np.linspace(-1, 1, T_line), y*np.ones(T_line)], axis=1))
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())



arr_pos, arr_color = stitch_patterns(arr_pos_list, arr_color_list)
print(arr_pos.shape, arr_color.shape)

arr_pos, arr_color = make_circular(arr_pos, arr_color)
print(arr_pos.shape, arr_color.shape)

dacpos_batch(arr_pos, arr_color, dac)

In [ ]:
# Make three concentric circles
T_line = 200
theta_line = np.linspace(0, 2*np.pi, T_line)

arr_pos_list = []
arr_color_list = []
for c_idx, scale in enumerate([0.9, 0.95, 1]):
    arr_pos_list.append(np.stack([np.cos(theta_line), np.sin(theta_line)], axis=1)/scale)
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())



arr_pos, arr_color = stitch_patterns(arr_pos_list, arr_color_list, angular_density=0)

dacpos_batch(arr_pos, arr_color, dac)

In [ ]:
draw_bitmap(arr_pos, arr_color)

In [ ]:
# make a sine wave
T_line = 1000
freq = 4
theta_line = np.linspace(0, 2*np.pi, T_line)
arr_pos = np.stack([np.linspace(-1, 1, T_line), np.sin(freq*theta_line)], axis=1)
arr_color = np.ones((T, 3))
arr_color[:, 0:2] = 0
arr_pos, arr_color = make_circular(arr_pos, arr_color)
dacpos_batch(arr_pos, arr_color, dac)